In [463]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stat
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import linear_model
%matplotlib inline
plt.style.use('fivethirtyeight')
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',50)
import os
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from catboost import CatBoostRegressor

In [464]:
print(os.getcwd())

/Users/abhayranjan/python-scripts/Club-Mahindra


In [465]:
train = pd.read_csv('/Users/abhayranjan/python-scripts/Club-Mahindra/train_5CLrC8b/train.csv')
train.head(n=5)

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,05/04/18,05/04/18,06/04/18,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,7.706428
1,03930f033646d073462b35d411616323597715ac4fc398...,23/01/15,11/04/15,16/04/15,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,6.662563
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,28/01/15,01/02/15,05/02/15,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,7.871602
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,02/05/15,11/06/15,16/06/15,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,5.344943
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,02/09/15,14/12/15,19/12/15,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,7.059346


In [466]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341424 entries, 0 to 341423
Data columns (total 24 columns):
reservation_id                        341424 non-null object
booking_date                          341424 non-null object
checkin_date                          341424 non-null object
checkout_date                         341424 non-null object
channel_code                          341424 non-null int64
main_product_code                     341424 non-null int64
numberofadults                        341424 non-null int64
numberofchildren                      341424 non-null int64
persontravellingid                    341424 non-null int64
resort_region_code                    341424 non-null int64
resort_type_code                      341424 non-null int64
room_type_booked_code                 341424 non-null int64
roomnights                            341424 non-null int64
season_holidayed_code                 341310 non-null float64
state_code_residence                  33666

In [467]:
train['booking_date'] = pd.to_datetime(train.booking_date,format='%d/%m/%y')
train['checkin_date'] = pd.to_datetime(train.checkin_date,format='%d/%m/%y')
train['checkout_date'] = pd.to_datetime(train.checkout_date,format='%d/%m/%y')
train['daysofstay'] = (train['checkout_date'] - train['checkin_date']).dt.days
train['checkout_year'] = train['checkout_date'].dt.year
train['checkout_month'] = train['checkout_date'].dt.month
train['checkout_dayofweek'] = train['checkout_date'].dt.dayofweek
train['checkout_week'] = train['checkout_date'].dt.week
train['checkout_quarter'] = train['checkout_date'].dt.quarter
train['checkin_year'] = train['checkin_date'].dt.year
train['checkin_month'] = train['checkin_date'].dt.month
train['checkin_dayofweek'] = train['checkin_date'].dt.dayofweek
train['checkin_week'] = train['checkin_date'].dt.week
train['checkin_quarter'] = train['checkin_date'].dt.quarter
train['booking_year'] = train['booking_date'].dt.year
train['booking_month'] = train['booking_date'].dt.month
train['booking_dayofweek'] = train['booking_date'].dt.dayofweek
train['booking_week'] = train['booking_date'].dt.week
train['booking_quarter'] = train['booking_date'].dt.quarter
train['booking_checkin_daysdifference'] = (train['checkin_date'] - train['booking_date']).dt.days
train['booking_checkout_daysdifference'] = (train['checkout_date'] - train['booking_date']).dt.days
train['booking_date_seconds'] = pd.to_datetime(train.booking_date).astype(int)/10**9
train['checkin_date_seconds'] = pd.to_datetime(train.checkin_date).astype(int)/10**9
train['checkout_date_seconds'] = pd.to_datetime(train.checkout_date).astype(int)/10**9
train.drop(['booking_date','checkin_date','checkout_date'],axis=1,inplace=True)
member_id_train = train.groupby(['memberid'],as_index=False)['reservation_id'].count().rename(columns={'reservation_id':'mem_res_count'})
resort_id_train = train.groupby('resort_id', as_index=False)['reservation_id'].count().rename(columns = {'reservation_id' : 'resort_id_counts'})
train = pd.merge(left=train,right=member_id_train,on='memberid',how='left')
train = pd.merge(left=train,right=resort_id_train,on='resort_id',how='left')
train['total_people_travel'] = train.numberofadults + train.numberofchildren
train['travel_in_group'] = train.numberofadults.apply(lambda x : 1 if x > 2 else 0)
train['travel_with_kids'] = train.numberofchildren.apply(lambda x : 1 if x > 0 else 0)
train['is_checkin_weekend'] = train.checkin_dayofweek.apply(lambda x:1 if (x==5 or x==6) else 0)
#train.head(n=50)

In [468]:
train.head(n=5)

,reservation_id,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled,daysofstay,checkout_year,checkout_month,checkout_dayofweek,checkout_week,checkout_quarter,checkin_year,checkin_month,checkin_dayofweek,checkin_week,checkin_quarter,booking_year,booking_month,booking_dayofweek,booking_week,booking_quarter,booking_checkin_daysdifference,booking_checkout_daysdifference,booking_date_seconds,checkin_date_seconds,checkout_date_seconds,mem_res_count,resort_id_counts,total_people_travel,travel_in_group,travel_with_kids,is_checkin_weekend
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,7.706428,1,2018,4,4,14,2,2018,4,3,14,2,2018,4,3,14,2,0,1,1.522886e+09,1.522886e+09,1.522973e+09,14,3879,2,0,0,0
1,03930f033646d073462b35d411616323597715ac4fc398...,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,6.662563,5,2015,4,3,16,2,2015,4,5,15,2,2015,1,4,4,1,78,83,1.421971e+09,1.428710e+09,1.429142e+09,14,5398,2,0,0,1
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,7.871602,4,2015,2,3,6,1,2015,2,6,5,1,2015,1,2,5,1,4,8,1.422403e+09,1.422749e+09,1.423094e+09,14,10168,2,0,0,1
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,5.344943,5,2015,6,1,25,2,2015,6,3,24,2,2015,5,5,18,2,40,45,1.430525e+09,1.433981e+09,1.434413e+09,14,35331,4,0,1,0
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,7.059346,5,2015,12,5,51,4,2015,12,0,51,4,2015,9,2,36,3,103,108,1.441152e+09,1.450051e+09,1.450483e+09,14,35331,2,0,0,0


In [469]:
train['state_code_residence'].fillna(train['state_code_residence'].mode()[0],inplace=True)
train['season_holidayed_code'].fillna(train['season_holidayed_code'].mode()[0],inplace=True)

In [470]:
train['season_holidayed_code'] = train['season_holidayed_code'].astype(np.int64)
train['state_code_residence'] = train['state_code_residence'].astype(np.int64)

In [471]:
(train.isnull().sum()/train.shape[0])*100

reservation_id                        0.0
channel_code                          0.0
main_product_code                     0.0
numberofadults                        0.0
numberofchildren                      0.0
persontravellingid                    0.0
resort_region_code                    0.0
resort_type_code                      0.0
room_type_booked_code                 0.0
roomnights                            0.0
season_holidayed_code                 0.0
state_code_residence                  0.0
state_code_resort                     0.0
total_pax                             0.0
member_age_buckets                    0.0
booking_type_code                     0.0
memberid                              0.0
cluster_code                          0.0
reservationstatusid_code              0.0
resort_id                             0.0
amount_spent_per_room_night_scaled    0.0
daysofstay                            0.0
checkout_year                         0.0
checkout_month                    

In [472]:
train.isnull().sum()

reservation_id                        0
channel_code                          0
main_product_code                     0
numberofadults                        0
numberofchildren                      0
persontravellingid                    0
resort_region_code                    0
resort_type_code                      0
room_type_booked_code                 0
roomnights                            0
season_holidayed_code                 0
state_code_residence                  0
state_code_resort                     0
total_pax                             0
member_age_buckets                    0
booking_type_code                     0
memberid                              0
cluster_code                          0
reservationstatusid_code              0
resort_id                             0
amount_spent_per_room_night_scaled    0
daysofstay                            0
checkout_year                         0
checkout_month                        0
checkout_dayofweek                    0


In [473]:
train['state_code_residence']

0          7
1          7
2          7
3          7
4          7
5          7
6          7
7          7
8          7
9          7
10         7
11         7
12         7
13         7
14         8
15         8
16         8
17         8
18         8
19         8
20         8
21         8
22         8
23         8
24         8
25         2
26         2
27         2
28         2
29         2
30         2
31         8
32         8
33         5
34         5
35         5
36         7
37         7
38         7
39         7
40         7
41         8
42         8
43         8
44         8
45         8
46         2
47         2
48         2
49         2
50         2
51         2
52         2
53         2
54         2
55         2
56         2
57         2
58         2
59         2
60         6
61         6
62         6
63         6
64         6
65         6
66         6
67         6
68         6
69         6
70         6
71         6
72         6
73         3
74         3
75         3
76         3

In [474]:
print(train.isnull().sum())

reservation_id                        0
channel_code                          0
main_product_code                     0
numberofadults                        0
numberofchildren                      0
persontravellingid                    0
resort_region_code                    0
resort_type_code                      0
room_type_booked_code                 0
roomnights                            0
season_holidayed_code                 0
state_code_residence                  0
state_code_resort                     0
total_pax                             0
member_age_buckets                    0
booking_type_code                     0
memberid                              0
cluster_code                          0
reservationstatusid_code              0
resort_id                             0
amount_spent_per_room_night_scaled    0
daysofstay                            0
checkout_year                         0
checkout_month                        0
checkout_dayofweek                    0


In [475]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341424 entries, 0 to 341423
Data columns (total 48 columns):
reservation_id                        341424 non-null object
channel_code                          341424 non-null int64
main_product_code                     341424 non-null int64
numberofadults                        341424 non-null int64
numberofchildren                      341424 non-null int64
persontravellingid                    341424 non-null int64
resort_region_code                    341424 non-null int64
resort_type_code                      341424 non-null int64
room_type_booked_code                 341424 non-null int64
roomnights                            341424 non-null int64
season_holidayed_code                 341424 non-null int64
state_code_residence                  341424 non-null int64
state_code_resort                     341424 non-null int64
total_pax                             341424 non-null int64
member_age_buckets                    341424 non

In [476]:
test = pd.read_csv('/Users/abhayranjan/python-scripts/Club-Mahindra/test.csv')
test.head(n=5)

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id
0,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d...,05/04/18,05/04/18,06/04/18,3,2,2,0,45,3,3,4,1,2.0,2.0,3,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,A,C,9f14025af0065b30e47e23ebb3b491d39ae8ed17d33739...
1,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb...,24/05/15,23/06/15,27/06/15,1,2,4,0,45,1,1,4,8,2.0,2.0,4,2,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,F,A,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a...
2,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e...,21/07/15,07/08/15,10/08/15,3,2,3,0,45,2,2,2,3,4.0,2.0,2,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...
3,09593c907ae262e46f655b4db9e14f54a19eadcfdd2679...,31/07/15,22/10/15,25/10/15,1,2,3,0,45,1,1,2,3,2.0,2.0,6,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,F,A,49d180ecf56132819571bf39d9b7b342522a2ac6d23c14...
4,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1...,01/07/16,01/10/16,05/10/16,1,2,4,0,45,1,2,3,8,2.0,2.0,11,2,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,F,A,e29c9c180c6279b0b02abd6a1801c7c04082cf486ec027...


In [477]:
test['booking_date'] = pd.to_datetime(test.booking_date,format='%d/%m/%y')
test['checkin_date'] = pd.to_datetime(test.checkin_date,format='%d/%m/%y')
test['checkout_date'] = pd.to_datetime(test.checkout_date,format='%d/%m/%y')
test['daysofstay'] = (test['checkout_date'] - test['checkin_date']).dt.days
test['checkout_year'] = test['checkout_date'].dt.year
test['checkout_month'] = test['checkout_date'].dt.month
test['checkout_dayofweek'] = test['checkout_date'].dt.dayofweek
test['checkout_week'] = test['checkout_date'].dt.week
test['checkout_quarter'] = test['checkout_date'].dt.quarter
test['checkin_year'] = test['checkin_date'].dt.year
test['checkin_month'] = test['checkin_date'].dt.month
test['checkin_dayofweek'] = test['checkin_date'].dt.dayofweek
test['checkin_week'] = test['checkin_date'].dt.week
test['checkin_quarter'] = test['checkin_date'].dt.quarter
test['booking_year'] = test['booking_date'].dt.year
test['booking_month'] = test['booking_date'].dt.month
test['booking_dayofweek'] = test['booking_date'].dt.dayofweek
test['booking_week'] = test['booking_date'].dt.week
test['booking_quarter'] = test['booking_date'].dt.quarter
test['booking_checkin_daysdifference'] = (test['checkin_date'] - test['booking_date']).dt.days
test['booking_checkout_daysdifference'] = (test['checkout_date'] - test['booking_date']).dt.days
test['booking_date_seconds'] = pd.to_datetime(test.booking_date).astype(int)/10**9
test['checkin_date_seconds'] = pd.to_datetime(test.checkin_date).astype(int)/10**9
test['checkout_date_seconds'] = pd.to_datetime(test.checkout_date).astype(int)/10**9
test.drop(['booking_date','checkin_date','checkout_date'],axis=1,inplace=True)
member_id_test = test.groupby(['memberid'],as_index=False)['reservation_id'].count().rename(columns={'reservation_id':'mem_res_count'})
resort_id_test = test.groupby('resort_id', as_index=False)['reservation_id'].count().rename(columns = {'reservation_id' : 'resort_id_counts'})
test = pd.merge(left=test,right=member_id_test,on='memberid',how='left')
test = pd.merge(left=test,right=resort_id_test,on='resort_id',how='left')
test['total_people_travel'] = test.numberofadults + test.numberofchildren
test['travel_in_group'] = test.numberofadults.apply(lambda x : 1 if x > 2 else 0)
test['travel_with_kids'] = test.numberofchildren.apply(lambda x : 1 if x > 0 else 0)
test['is_checkin_weekend'] = test.checkin_dayofweek.apply(lambda x:1 if (x==5 or x==6) else 0)
#test.head(n=50)

In [478]:
test.head(n=5)

,reservation_id,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,daysofstay,checkout_year,checkout_month,checkout_dayofweek,checkout_week,checkout_quarter,checkin_year,checkin_month,checkin_dayofweek,checkin_week,checkin_quarter,booking_year,booking_month,booking_dayofweek,booking_week,booking_quarter,booking_checkin_daysdifference,booking_checkout_daysdifference,booking_date_seconds,checkin_date_seconds,checkout_date_seconds,mem_res_count,resort_id_counts,total_people_travel,travel_in_group,travel_with_kids,is_checkin_weekend
0,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d...,3,2,2,0,45,3,3,4,1,2.0,2.0,3,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,A,C,9f14025af0065b30e47e23ebb3b491d39ae8ed17d33739...,1,2018,4,4,14,2,2018,4,3,14,2,2018,4,3,14,2,0,1,1.522886e+09,1.522886e+09,1.522973e+09,7,3863,2,0,0,0
1,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb...,1,2,4,0,45,1,1,4,8,2.0,2.0,4,2,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,F,A,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a...,4,2015,6,5,26,2,2015,6,1,26,2,2015,5,6,21,2,30,34,1.432426e+09,1.435018e+09,1.435363e+09,7,5189,4,1,0,0
2,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e...,3,2,3,0,45,2,2,2,3,4.0,2.0,2,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,3,2015,8,0,33,3,2015,8,4,32,3,2015,7,1,30,3,17,20,1.437437e+09,1.438906e+09,1.439165e+09,7,15397,3,1,0,0
3,09593c907ae262e46f655b4db9e14f54a19eadcfdd2679...,1,2,3,0,45,1,1,2,3,2.0,2.0,6,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,F,A,49d180ecf56132819571bf39d9b7b342522a2ac6d23c14...,3,2015,10,6,43,4,2015,10,3,43,4,2015,7,4,31,3,83,86,1.438301e+09,1.445472e+09,1.445731e+09,7,9563,3,1,0,0
4,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1...,1,2,4,0,45,1,2,3,8,2.0,2.0,11,2,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d84...,F,A,e29c9c180c6279b0b02abd6a1801c7c04082cf486ec027...,4,2016,10,2,40,4,2016,10,5,39,4,2016,7,4,26,3,92,96,1.467331e+09,1.475280e+09,1.475626e+09,7,7909,4,1,0,1


In [479]:
categorical_features =['booking_dayofweek','booking_month','booking_type_code','booking_year','booking_week','booking_quarter','channel_code','checkin_dayofweek','checkin_month','checkin_year','checkin_week','checkin_quarter','checkout_dayofweek','checkout_month','checkout_year','checkout_week','checkout_quarter','cluster_code','reservationstatusid_code','main_product_code','member_age_buckets','resort_region_code','resort_type_code','room_type_booked_code','season_holidayed_code','state_code_residence','state_code_resort','travel_in_group','travel_with_kids','is_checkin_weekend']

In [480]:
cols = [col for col in train.columns if col not in (['amount_spent_per_room_night_scaled','reservation_id','memberid'])]
#print(cols) 
X = train[cols]
y = train['amount_spent_per_room_night_scaled']
X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.20,random_state=99)

In [481]:
#print(X.head(n=5))
#print(y.head(n=5))

In [482]:
columns_dict = {}
for i,j in enumerate(cols):
    columns_dict[j]=i
print(columns_dict)
categorical_columns_index_list = []
for i,j in columns_dict.items():
    if (i in categorical_features):
        categorical_columns_index_list.append(j)
#print(categorical_columns_index_list)

{'channel_code': 0, 'main_product_code': 1, 'numberofadults': 2, 'numberofchildren': 3, 'persontravellingid': 4, 'resort_region_code': 5, 'resort_type_code': 6, 'room_type_booked_code': 7, 'roomnights': 8, 'season_holidayed_code': 9, 'state_code_residence': 10, 'state_code_resort': 11, 'total_pax': 12, 'member_age_buckets': 13, 'booking_type_code': 14, 'cluster_code': 15, 'reservationstatusid_code': 16, 'resort_id': 17, 'daysofstay': 18, 'checkout_year': 19, 'checkout_month': 20, 'checkout_dayofweek': 21, 'checkout_week': 22, 'checkout_quarter': 23, 'checkin_year': 24, 'checkin_month': 25, 'checkin_dayofweek': 26, 'checkin_week': 27, 'checkin_quarter': 28, 'booking_year': 29, 'booking_month': 30, 'booking_dayofweek': 31, 'booking_week': 32, 'booking_quarter': 33, 'booking_checkin_daysdifference': 34, 'booking_checkout_daysdifference': 35, 'booking_date_seconds': 36, 'checkin_date_seconds': 37, 'checkout_date_seconds': 38, 'mem_res_count': 39, 'resort_id_counts': 40, 'total_people_trave

In [483]:
#print(X_train.head(n=5))
#print(X_train.info())

In [484]:
#cbreg = CatBoostRegressor(iterations=700,learning_rate=0.02,random_seed=23,depth=12,eval_metric='RMSE',metric_period=75,od_wait=100)
#cbreg.fit(X_train,y_train,eval_set=(X_valid,y_valid),cat_features=categorical_columns_index_list,use_best_model=True,verbose=True)

In [485]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
for cols in categorical_features:
    ohe_train = pd.get_dummies(train[cols],prefix=cols,drop_first=True)
    #print(ohe_train)
    train = pd.concat((train,ohe_train),axis=1)
    train.drop(cols,axis=1,inplace=True)

In [486]:
#print(train.columns)

In [487]:
#train.head(n=5)

In [488]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
for cols in categorical_features:
    ohe_test = pd.get_dummies(test[cols],prefix=cols,drop_first=True)
    test = pd.concat((test,ohe_test),axis=1)
    test.drop(cols,axis=1,inplace=True)

In [489]:
#print(test.columns)

In [490]:
test_cols = [col for col in test.columns if col not in (['reservation_id','memberid','resort_id'])]
test = test[test_cols]
X_test = np.array(test)

In [491]:
"""params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'max_depth': 6, 
    'learning_rate': 0.02,
    'verbose': 0, 
    'early_stopping_round': 500}
n_estimators = 100

n_iters = 5
preds_buf = []
err_buf = []"""

"params = {\n    'boosting_type': 'gbdt',\n    'objective': 'regression',\n    'metric': 'rmse',\n    'max_depth': 6, \n    'learning_rate': 0.02,\n    'verbose': 0, \n    'early_stopping_round': 500}\nn_estimators = 100\n\nn_iters = 5\npreds_buf = []\nerr_buf = []"

In [492]:
param = {
    'num_leaves':10,
    'boosting_type':'gbdt',
    'objective':'regression',
    'max_bin':119,
    'min_data_in_leaf':11,
    'learning_rate':0.02,
    'bagging_fraction':1.0,
    'bagging_freq':5,
    'feature_fraction':0.05,
    'min_gain_to_split':0.65,
    'max_depth':32,
    'save_binary':True,
    'boosting_type':'gbdt',
    'metric':'rmse',
    'verbose':0,
    'is_unbalanced':True,    
}
n_estimators = 100

n_iters = 5
preds_buf = []
err_buf = []

In [493]:
#X.dtypes

In [494]:
import lightgbm as lgb
cols = [col for col in train.columns if col not in (['amount_spent_per_room_night_scaled','reservation_id','memberid','resort_id'])] 
X = train[cols]
y = train['amount_spent_per_room_night_scaled']
preds = []
for i in range(n_iters): 
    x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.10, random_state=9)
    d_train = lgb.Dataset(x_train, label=y_train)
    d_valid = lgb.Dataset(x_valid, label=y_valid)
    watchlist = [d_valid]

    model = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

    preds = model.predict(x_valid)
    #err = rmse(y_valid, preds)
    #err_buf.append(err)
    #print('RMSE = ' + str(err))
    
    preds = model.predict(X_test)
    preds_buf.append(preds)
    print(model.feature_importance)
preds = np.mean(preds_buf, axis=0)

#print(model.feature_importance)

/Users/abhayranjan/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's rmse: 1.0806
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's rmse: 1.07797
[3]	valid_0's rmse: 1.07544
[4]	valid_0's rmse: 1.07301
[5]	valid_0's rmse: 1.07067
[6]	valid_0's rmse: 1.06841
[7]	valid_0's rmse: 1.06623
[8]	valid_0's rmse: 1.06411
[9]	valid_0's rmse: 1.06206
[10]	valid_0's rmse: 1.06009
[11]	valid_0's rmse: 1.05815
[12]	valid_0's rmse: 1.05634
[13]	valid_0's rmse: 1.05457
[14]	valid_0's rmse: 1.05286
[15]	valid_0's rmse: 1.05117
[16]	valid_0's rmse: 1.04955
[17]	valid_0's rmse: 1.04801
[18]	valid_0's rmse: 1.04648
[19]	valid_0's rmse: 1.04506
[20]	valid_0's rmse: 1.04363
[21]	valid_0's rmse: 1.04224
[22]	valid_0's rmse: 1.0409
[23]	valid_0's rmse: 1.03964
[24]	valid_0's rmse: 1.03844
[25]	valid_0's rmse: 1.03723
[26]	valid_0's rmse: 1.03606
[27]	valid_0's rmse: 1.03497
[28]	valid_0's rmse: 1.03387
[29]	valid_0's rmse: 1.0328
[30]	valid_0's rmse: 1.03174
[31]	valid_0's rmse: 1.03075
[32]	valid_0's rmse: 1.02979
[33]	valid_0's rmse: 

/Users/abhayranjan/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's rmse: 1.0806
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's rmse: 1.07797
[3]	valid_0's rmse: 1.07544
[4]	valid_0's rmse: 1.07301
[5]	valid_0's rmse: 1.07067
[6]	valid_0's rmse: 1.06841
[7]	valid_0's rmse: 1.06623
[8]	valid_0's rmse: 1.06411
[9]	valid_0's rmse: 1.06206
[10]	valid_0's rmse: 1.06009
[11]	valid_0's rmse: 1.05815
[12]	valid_0's rmse: 1.05634
[13]	valid_0's rmse: 1.05457
[14]	valid_0's rmse: 1.05286
[15]	valid_0's rmse: 1.05117
[16]	valid_0's rmse: 1.04955
[17]	valid_0's rmse: 1.04801
[18]	valid_0's rmse: 1.04648
[19]	valid_0's rmse: 1.04506
[20]	valid_0's rmse: 1.04363
[21]	valid_0's rmse: 1.04224
[22]	valid_0's rmse: 1.0409
[23]	valid_0's rmse: 1.03964
[24]	valid_0's rmse: 1.03844
[25]	valid_0's rmse: 1.03723
[26]	valid_0's rmse: 1.03606
[27]	valid_0's rmse: 1.03497
[28]	valid_0's rmse: 1.03387
[29]	valid_0's rmse: 1.0328
[30]	valid_0's rmse: 1.03174
[31]	valid_0's rmse: 1.03075
[32]	valid_0's rmse: 1.02979
[33]	valid_0's rmse: 

/Users/abhayranjan/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's rmse: 1.0806
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's rmse: 1.07797
[3]	valid_0's rmse: 1.07544
[4]	valid_0's rmse: 1.07301
[5]	valid_0's rmse: 1.07067
[6]	valid_0's rmse: 1.06841
[7]	valid_0's rmse: 1.06623
[8]	valid_0's rmse: 1.06411
[9]	valid_0's rmse: 1.06206
[10]	valid_0's rmse: 1.06009
[11]	valid_0's rmse: 1.05815
[12]	valid_0's rmse: 1.05634
[13]	valid_0's rmse: 1.05457
[14]	valid_0's rmse: 1.05286
[15]	valid_0's rmse: 1.05117
[16]	valid_0's rmse: 1.04955
[17]	valid_0's rmse: 1.04801
[18]	valid_0's rmse: 1.04648
[19]	valid_0's rmse: 1.04506
[20]	valid_0's rmse: 1.04363
[21]	valid_0's rmse: 1.04224
[22]	valid_0's rmse: 1.0409
[23]	valid_0's rmse: 1.03964
[24]	valid_0's rmse: 1.03844
[25]	valid_0's rmse: 1.03723
[26]	valid_0's rmse: 1.03606
[27]	valid_0's rmse: 1.03497
[28]	valid_0's rmse: 1.03387
[29]	valid_0's rmse: 1.0328
[30]	valid_0's rmse: 1.03174
[31]	valid_0's rmse: 1.03075
[32]	valid_0's rmse: 1.02979
[33]	valid_0's rmse: 

/Users/abhayranjan/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's rmse: 1.0806
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's rmse: 1.07797
[3]	valid_0's rmse: 1.07544
[4]	valid_0's rmse: 1.07301
[5]	valid_0's rmse: 1.07067
[6]	valid_0's rmse: 1.06841
[7]	valid_0's rmse: 1.06623
[8]	valid_0's rmse: 1.06411
[9]	valid_0's rmse: 1.06206
[10]	valid_0's rmse: 1.06009
[11]	valid_0's rmse: 1.05815
[12]	valid_0's rmse: 1.05634
[13]	valid_0's rmse: 1.05457
[14]	valid_0's rmse: 1.05286
[15]	valid_0's rmse: 1.05117
[16]	valid_0's rmse: 1.04955
[17]	valid_0's rmse: 1.04801
[18]	valid_0's rmse: 1.04648
[19]	valid_0's rmse: 1.04506
[20]	valid_0's rmse: 1.04363
[21]	valid_0's rmse: 1.04224
[22]	valid_0's rmse: 1.0409
[23]	valid_0's rmse: 1.03964
[24]	valid_0's rmse: 1.03844
[25]	valid_0's rmse: 1.03723
[26]	valid_0's rmse: 1.03606
[27]	valid_0's rmse: 1.03497
[28]	valid_0's rmse: 1.03387
[29]	valid_0's rmse: 1.0328
[30]	valid_0's rmse: 1.03174
[31]	valid_0's rmse: 1.03075
[32]	valid_0's rmse: 1.02979
[33]	valid_0's rmse: 

/Users/abhayranjan/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's rmse: 1.0806
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's rmse: 1.07797
[3]	valid_0's rmse: 1.07544
[4]	valid_0's rmse: 1.07301
[5]	valid_0's rmse: 1.07067
[6]	valid_0's rmse: 1.06841
[7]	valid_0's rmse: 1.06623
[8]	valid_0's rmse: 1.06411
[9]	valid_0's rmse: 1.06206
[10]	valid_0's rmse: 1.06009
[11]	valid_0's rmse: 1.05815
[12]	valid_0's rmse: 1.05634
[13]	valid_0's rmse: 1.05457
[14]	valid_0's rmse: 1.05286
[15]	valid_0's rmse: 1.05117
[16]	valid_0's rmse: 1.04955
[17]	valid_0's rmse: 1.04801
[18]	valid_0's rmse: 1.04648
[19]	valid_0's rmse: 1.04506
[20]	valid_0's rmse: 1.04363
[21]	valid_0's rmse: 1.04224
[22]	valid_0's rmse: 1.0409
[23]	valid_0's rmse: 1.03964
[24]	valid_0's rmse: 1.03844
[25]	valid_0's rmse: 1.03723
[26]	valid_0's rmse: 1.03606
[27]	valid_0's rmse: 1.03497
[28]	valid_0's rmse: 1.03387
[29]	valid_0's rmse: 1.0328
[30]	valid_0's rmse: 1.03174
[31]	valid_0's rmse: 1.03075
[32]	valid_0's rmse: 1.02979
[33]	valid_0's rmse: 

In [495]:
feature_score = pd.DataFrame(model.feature_importance(), columns=['score'])
feature_score['feature'] = cols
feature_score = feature_score.sort_values(by=['score'], ascending=False)
feature_score.to_csv('feature_score.csv', index=False)

In [496]:
"""features = [col for col in train.columns if col not in (['amount_spent_per_room_night_scaled','reservation_id','memberid','resort_id'])] 
X = train[features]
y = train['amount_spent_per_room_night_scaled']
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100,n_jobs=4,max_depth=6)
boruta_selector  = BorutaPy(rfc,n_estimators='auto',verbose=2)
boruta_selector.fit(X,y)
print(boruta_selector.support_)
print(boruta_selector.ranking_)
X_mod = X[features]
selected_features = list(X.columns[boruta_selector.support_])
print("******")
print(selected_features)"""

'features = [col for col in train.columns if col not in ([\'amount_spent_per_room_night_scaled\',\'reservation_id\',\'memberid\',\'resort_id\'])] \nX = train[features]\ny = train[\'amount_spent_per_room_night_scaled\']\nfrom boruta import BorutaPy\nfrom sklearn.ensemble import RandomForestClassifier\nrfc = RandomForestClassifier(n_estimators=100,n_jobs=4,max_depth=6)\nboruta_selector  = BorutaPy(rfc,n_estimators=\'auto\',verbose=2)\nboruta_selector.fit(X,y)\nprint(boruta_selector.support_)\nprint(boruta_selector.ranking_)\nX_mod = X[features]\nselected_features = list(X.columns[boruta_selector.support_])\nprint("******")\nprint(selected_features)'

In [497]:
subm = pd.DataFrame()
subm['reservation_id'] = pd.read_csv('/Users/abhayranjan/python-scripts/Club-Mahindra/test.csv').reservation_id.values
print(len(subm['reservation_id']))
print(len(preds))
subm['amount_spent_per_room_night_scaled'] = preds
subm.to_csv('submission_lgbm5.csv', index=False)
print(os.getcwd())

146765
146765
/Users/abhayranjan/python-scripts/Club-Mahindra
